In [89]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop

import tensorflow.compat as tf
#import tensorflow.compat.v1 as tf 
#tf.disable_v2_behavior()

from tensorflow import keras
from tensorflow.keras import layers

import csv

from sklearn.utils import shuffle

from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.layers.core import Dropout

In [90]:
# citire imagini de antrenare plus etichete de antrenare
f = open ("train.txt")
train_images = []
train_labels = []
for line in enumerate(open("train.txt", "r")):
    v = line[1].split(',')
    image_name = 'train/'+v[0]
    label = int(v[1])
    train_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    train_images.append(image)
# transformare în nparray
train_images = np.array(train_images)
train_labels = np.array(train_labels)
train_images, train_labels = shuffle(train_images, train_labels, random_state=0)

In [91]:
# citire imagini de testare 
f = open ("test.txt")
test_images = []
name_test_images =[]
for line in enumerate(open("test.txt", "r")):
    image_name = 'test/'+ line[1][:len(line[1])-1]
    name_test_images.append(line[1][:len(line[1])-1])
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    test_images.append(image)
# transformare în nparray
test_images = np.array(test_images)

In [92]:
# citire imagini de validare plus etichete de validare
f = open ("validation.txt")
validation_images = []
validation_labels = []
for line in enumerate(open("validation.txt", "r")):
    v = line[1].split(',')
    image_name = 'validation/'+v[0]
    label = int(v[1])
    validation_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    validation_images.append(image)
# transformare în nparray
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)
validation_images, validation_labels = shuffle(validation_images, validation_labels, random_state=0)

In [93]:
nb_classes = 9 #9 categorii notate de la 0 la 8

x_train=train_images.astype('float32')
x_val=validation_images.astype('float32')
test_images=test_images.astype('float32')

y_val=np_utils.to_categorical(validation_labels, nb_classes) 
y_train=np_utils.to_categorical(train_labels, nb_classes) 

In [94]:
# Marire esantion de date
# creez un esantion separat
# prin aplicarea de transformari aleatorii imaginilor de antrenament 
# Scop: evitarea overfitting-ului & expunerea modelului la mai multe cazuri de studiu  
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"), #rasucire orizontala
        layers.experimental.preprocessing.RandomRotation(0.1), #rotatie aleatorie a imaginii
    ]
)
# creare model de baza
base_model = keras.applications.Xception(
    weights="imagenet",  # incarcare date preantrenate din ImageNet
    input_shape=(150, 150, 3), #dimensiuni ImageNet
    include_top=False
)  


# "Dezactivare" model de baza (Freezing) 
base_model.trainable = False

# Craere model nou 
inputs = keras.Input(shape=(32, 32, 3)) 
x = data_augmentation(inputs)  # Aplicare transformari aleatorii 

# Adaug un layer de normalizare pe intervalele [0,255] & [-1,1] => datele preantrenate au nevoie de normalizare
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# [-1,1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# "Batch normalization" -> model mai rapid si stabil, normalizare layer de intrare (prin recentrare si redimensionare)
# Modelul de baza contine layere batch-norm 
# Pastrare model in modul de inferenta (inference mode) -> pregatirea pentru fine-tuning  
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

# Pentru regularizare adaug un Dropout layer => Scop: reducere overfitting
x = keras.layers.Dropout(0.2)(x)  

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

#model.summary()

In [96]:
model = Sequential()
model.add(Dense(1000, input_shape =(x_train.shape[1],)))
model.add(Activation ('relu'))
model.add(Dense(200))
model.add(Activation ('sigmoid'))
model.add(Dense(nb_classes))
model.add(Activation ('softmax'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-6, decay=1e-6), metrics=['accuracy'])
history= model.fit (x_train, y_train, batch_size=200, epochs=100, verbose=2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)

Epoch 1/1000
151/151 - 4s - loss: 0.2922 - accuracy: 0.3744
Epoch 2/1000
151/151 - 3s - loss: 0.2412 - accuracy: 0.5122
Epoch 3/1000
151/151 - 3s - loss: 0.2194 - accuracy: 0.5644
Epoch 4/1000
151/151 - 3s - loss: 0.2040 - accuracy: 0.6022
Epoch 5/1000
151/151 - 3s - loss: 0.1940 - accuracy: 0.6230
Epoch 6/1000
151/151 - 3s - loss: 0.1837 - accuracy: 0.6472
Epoch 7/1000
151/151 - 3s - loss: 0.1768 - accuracy: 0.6618
Epoch 8/1000
151/151 - 3s - loss: 0.1693 - accuracy: 0.6769
Epoch 9/1000
151/151 - 3s - loss: 0.1633 - accuracy: 0.6929
Epoch 10/1000
151/151 - 3s - loss: 0.1578 - accuracy: 0.7015
Epoch 11/1000
151/151 - 3s - loss: 0.1516 - accuracy: 0.7191
Epoch 12/1000
151/151 - 3s - loss: 0.1478 - accuracy: 0.7244
Epoch 13/1000
151/151 - 3s - loss: 0.1415 - accuracy: 0.7390
Epoch 14/1000
151/151 - 4s - loss: 0.1382 - accuracy: 0.7468
Epoch 15/1000
151/151 - 4s - loss: 0.1333 - accuracy: 0.7563
Epoch 16/1000
151/151 - 4s - loss: 0.1290 - accuracy: 0.7661
Epoch 17/1000
151/151 - 4s - loss

Epoch 135/1000
151/151 - 4s - loss: 0.0131 - accuracy: 0.9849
Epoch 136/1000
151/151 - 4s - loss: 0.0128 - accuracy: 0.9847
Epoch 137/1000
151/151 - 4s - loss: 0.0114 - accuracy: 0.9865
Epoch 138/1000
151/151 - 4s - loss: 0.0126 - accuracy: 0.9849
Epoch 139/1000
151/151 - 4s - loss: 0.0111 - accuracy: 0.9872
Epoch 140/1000
151/151 - 4s - loss: 0.0117 - accuracy: 0.9867
Epoch 141/1000
151/151 - 5s - loss: 0.0108 - accuracy: 0.9879
Epoch 142/1000
151/151 - 4s - loss: 0.0119 - accuracy: 0.9857
Epoch 143/1000
151/151 - 5s - loss: 0.0113 - accuracy: 0.9871
Epoch 144/1000
151/151 - 4s - loss: 0.0126 - accuracy: 0.9850
Epoch 145/1000
151/151 - 4s - loss: 0.0116 - accuracy: 0.9865
Epoch 146/1000
151/151 - 4s - loss: 0.0122 - accuracy: 0.9866
Epoch 147/1000
151/151 - 5s - loss: 0.0124 - accuracy: 0.9854
Epoch 148/1000
151/151 - 5s - loss: 0.0114 - accuracy: 0.9860
Epoch 149/1000
151/151 - 5s - loss: 0.0119 - accuracy: 0.9867
Epoch 150/1000
151/151 - 4s - loss: 0.0107 - accuracy: 0.9866
Epoch 15

151/151 - 4s - loss: 0.0069 - accuracy: 0.9929
Epoch 268/1000
151/151 - 4s - loss: 0.0060 - accuracy: 0.9937
Epoch 269/1000
151/151 - 4s - loss: 0.0067 - accuracy: 0.9931
Epoch 270/1000
151/151 - 4s - loss: 0.0067 - accuracy: 0.9929
Epoch 271/1000
151/151 - 4s - loss: 0.0073 - accuracy: 0.9931
Epoch 272/1000
151/151 - 4s - loss: 0.0063 - accuracy: 0.9934
Epoch 273/1000
151/151 - 4s - loss: 0.0067 - accuracy: 0.9929
Epoch 274/1000
151/151 - 4s - loss: 0.0056 - accuracy: 0.9945
Epoch 275/1000
151/151 - 4s - loss: 0.0059 - accuracy: 0.9937
Epoch 276/1000
151/151 - 4s - loss: 0.0060 - accuracy: 0.9939
Epoch 277/1000
151/151 - 4s - loss: 0.0074 - accuracy: 0.9928
Epoch 278/1000
151/151 - 4s - loss: 0.0074 - accuracy: 0.9923
Epoch 279/1000
151/151 - 4s - loss: 0.0064 - accuracy: 0.9934
Epoch 280/1000
151/151 - 4s - loss: 0.0069 - accuracy: 0.9928
Epoch 281/1000
151/151 - 4s - loss: 0.0061 - accuracy: 0.9935
Epoch 282/1000
151/151 - 4s - loss: 0.0059 - accuracy: 0.9939
Epoch 283/1000
151/151 

Epoch 400/1000
151/151 - 5s - loss: 0.0053 - accuracy: 0.9946
Epoch 401/1000
151/151 - 5s - loss: 0.0056 - accuracy: 0.9946
Epoch 402/1000
151/151 - 6s - loss: 0.0058 - accuracy: 0.9941
Epoch 403/1000
151/151 - 5s - loss: 0.0053 - accuracy: 0.9949
Epoch 404/1000
151/151 - 5s - loss: 0.0052 - accuracy: 0.9950
Epoch 405/1000
151/151 - 5s - loss: 0.0046 - accuracy: 0.9954
Epoch 406/1000
151/151 - 5s - loss: 0.0052 - accuracy: 0.9952
Epoch 407/1000
151/151 - 5s - loss: 0.0058 - accuracy: 0.9947
Epoch 408/1000
151/151 - 5s - loss: 0.0050 - accuracy: 0.9952
Epoch 409/1000
151/151 - 5s - loss: 0.0055 - accuracy: 0.9950
Epoch 410/1000
151/151 - 5s - loss: 0.0057 - accuracy: 0.9943
Epoch 411/1000
151/151 - 5s - loss: 0.0053 - accuracy: 0.9950
Epoch 412/1000
151/151 - 5s - loss: 0.0048 - accuracy: 0.9952
Epoch 413/1000
151/151 - 6s - loss: 0.0053 - accuracy: 0.9952
Epoch 414/1000
151/151 - 6s - loss: 0.0051 - accuracy: 0.9955
Epoch 415/1000
151/151 - 6s - loss: 0.0051 - accuracy: 0.9951
Epoch 41

151/151 - 5s - loss: 0.0041 - accuracy: 0.9965
Epoch 533/1000
151/151 - 5s - loss: 0.0039 - accuracy: 0.9966
Epoch 534/1000
151/151 - 5s - loss: 0.0040 - accuracy: 0.9967
Epoch 535/1000
151/151 - 5s - loss: 0.0042 - accuracy: 0.9962
Epoch 536/1000
151/151 - 5s - loss: 0.0046 - accuracy: 0.9958
Epoch 537/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9965
Epoch 538/1000
151/151 - 5s - loss: 0.0049 - accuracy: 0.9950
Epoch 539/1000
151/151 - 5s - loss: 0.0036 - accuracy: 0.9966
Epoch 540/1000
151/151 - 5s - loss: 0.0040 - accuracy: 0.9966
Epoch 541/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9962
Epoch 542/1000
151/151 - 5s - loss: 0.0033 - accuracy: 0.9970
Epoch 543/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9963
Epoch 544/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9962
Epoch 545/1000
151/151 - 5s - loss: 0.0047 - accuracy: 0.9961
Epoch 546/1000
151/151 - 6s - loss: 0.0045 - accuracy: 0.9958
Epoch 547/1000
151/151 - 5s - loss: 0.0039 - accuracy: 0.9964
Epoch 548/1000
151/151 

Epoch 665/1000
151/151 - 5s - loss: 0.0039 - accuracy: 0.9966
Epoch 666/1000
151/151 - 5s - loss: 0.0038 - accuracy: 0.9967
Epoch 667/1000
151/151 - 5s - loss: 0.0034 - accuracy: 0.9970
Epoch 668/1000
151/151 - 5s - loss: 0.0035 - accuracy: 0.9970
Epoch 669/1000
151/151 - 5s - loss: 0.0038 - accuracy: 0.9967
Epoch 670/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9963
Epoch 671/1000
151/151 - 5s - loss: 0.0043 - accuracy: 0.9965
Epoch 672/1000
151/151 - 5s - loss: 0.0049 - accuracy: 0.9959
Epoch 673/1000
151/151 - 5s - loss: 0.0038 - accuracy: 0.9965
Epoch 674/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9964
Epoch 675/1000
151/151 - 5s - loss: 0.0046 - accuracy: 0.9959
Epoch 676/1000
151/151 - 5s - loss: 0.0036 - accuracy: 0.9967
Epoch 677/1000
151/151 - 5s - loss: 0.0036 - accuracy: 0.9968
Epoch 678/1000
151/151 - 5s - loss: 0.0037 - accuracy: 0.9970
Epoch 679/1000
151/151 - 5s - loss: 0.0041 - accuracy: 0.9965
Epoch 680/1000
151/151 - 5s - loss: 0.0038 - accuracy: 0.9967
Epoch 68

151/151 - 5s - loss: 0.0032 - accuracy: 0.9973
Epoch 798/1000
151/151 - 5s - loss: 0.0035 - accuracy: 0.9969
Epoch 799/1000
151/151 - 5s - loss: 0.0036 - accuracy: 0.9970
Epoch 800/1000
151/151 - 5s - loss: 0.0035 - accuracy: 0.9970
Epoch 801/1000
151/151 - 5s - loss: 0.0039 - accuracy: 0.9969
Epoch 802/1000
151/151 - 5s - loss: 0.0040 - accuracy: 0.9968
Epoch 803/1000
151/151 - 5s - loss: 0.0031 - accuracy: 0.9976
Epoch 804/1000
151/151 - 5s - loss: 0.0035 - accuracy: 0.9970
Epoch 805/1000
151/151 - 5s - loss: 0.0037 - accuracy: 0.9970
Epoch 806/1000
151/151 - 5s - loss: 0.0037 - accuracy: 0.9970
Epoch 807/1000
151/151 - 5s - loss: 0.0034 - accuracy: 0.9971
Epoch 808/1000
151/151 - 5s - loss: 0.0037 - accuracy: 0.9969
Epoch 809/1000
151/151 - 5s - loss: 0.0032 - accuracy: 0.9975
Epoch 810/1000
151/151 - 5s - loss: 0.0040 - accuracy: 0.9969
Epoch 811/1000
151/151 - 5s - loss: 0.0036 - accuracy: 0.9970
Epoch 812/1000
151/151 - 5s - loss: 0.0036 - accuracy: 0.9973
Epoch 813/1000
151/151 

Epoch 930/1000
151/151 - 5s - loss: 0.0030 - accuracy: 0.9973
Epoch 931/1000
151/151 - 5s - loss: 0.0038 - accuracy: 0.9968
Epoch 932/1000
151/151 - 5s - loss: 0.0032 - accuracy: 0.9972
Epoch 933/1000
151/151 - 5s - loss: 0.0030 - accuracy: 0.9975
Epoch 934/1000
151/151 - 5s - loss: 0.0029 - accuracy: 0.9976
Epoch 935/1000
151/151 - 5s - loss: 0.0034 - accuracy: 0.9974
Epoch 936/1000
151/151 - 5s - loss: 0.0032 - accuracy: 0.9975
Epoch 937/1000
151/151 - 5s - loss: 0.0025 - accuracy: 0.9983
Epoch 938/1000
151/151 - 5s - loss: 0.0031 - accuracy: 0.9974
Epoch 939/1000
151/151 - 5s - loss: 0.0032 - accuracy: 0.9977
Epoch 940/1000
151/151 - 5s - loss: 0.0037 - accuracy: 0.9968
Epoch 941/1000
151/151 - 5s - loss: 0.0028 - accuracy: 0.9975
Epoch 942/1000
151/151 - 5s - loss: 0.0035 - accuracy: 0.9970
Epoch 943/1000
151/151 - 5s - loss: 0.0034 - accuracy: 0.9971
Epoch 944/1000
151/151 - 5s - loss: 0.0032 - accuracy: 0.9973
Epoch 945/1000
151/151 - 5s - loss: 0.0035 - accuracy: 0.9971
Epoch 94

In [97]:
# Antrenare model principal
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)


In [98]:
# "Fine tunning"
# "Activare" (Unfreeze) model de baza aflat in "inference mode"

base_model.trainable = True
model.summary()


model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # rata de invatare mica
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history= model.fit (x_train, y_train, batch_size=200, epochs=20, verbose=2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 1000)              1025000   
_________________________________________________________________
activation_24 (Activation)   (None, 1000)              0         
_________________________________________________________________
dense_58 (Dense)             (None, 200)               200200    
_________________________________________________________________
activation_25 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 9)                 1809      
_________________________________________________________________
activation_26 (Activation)   (None, 9)                 0         
Total params: 1,227,009
Trainable params: 1,227,009
Non-trainable params: 0
___________________________________________

In [99]:
predict =model.predict_classes(test_images).astype("int32")
f = open("submission_2.csv", "w", newline="")
csv_file= csv.writer(f, delimiter=",")
csv_file.writerow(['id', 'label'])
for i in range (predict.shape[0]):
    print(name_test_images[i], ",", predict[i], sep="")
    csv_file.writerow([name_test_images[i], predict[i]])
f.close()

035001.png,5
035002.png,0
035003.png,7
035004.png,5
035005.png,4
035006.png,0
035007.png,0
035008.png,7
035009.png,7
035010.png,7
035011.png,1
035012.png,4
035013.png,5
035014.png,1
035015.png,4
035016.png,7
035017.png,1
035018.png,4
035019.png,1
035020.png,6
035021.png,0
035022.png,0
035023.png,5
035024.png,8
035025.png,4
035026.png,4
035027.png,3
035028.png,5
035029.png,8
035030.png,2
035031.png,5
035032.png,0
035033.png,0
035034.png,1
035035.png,3
035036.png,7
035037.png,2
035038.png,6
035039.png,6
035040.png,0
035041.png,3
035042.png,6
035043.png,0
035044.png,0
035045.png,8
035046.png,5
035047.png,7
035048.png,3
035049.png,3
035050.png,3
035051.png,0
035052.png,3
035053.png,7
035054.png,5
035055.png,1
035056.png,7
035057.png,7
035058.png,5
035059.png,4
035060.png,1
035061.png,8
035062.png,4
035063.png,6
035064.png,1
035065.png,3
035066.png,5
035067.png,4
035068.png,1
035069.png,7
035070.png,1
035071.png,3
035072.png,1
035073.png,7
035074.png,1
035075.png,6
035076.png,4
035077.png,0

035703.png,2
035704.png,2
035705.png,5
035706.png,4
035707.png,7
035708.png,1
035709.png,5
035710.png,2
035711.png,1
035712.png,7
035713.png,8
035714.png,8
035715.png,2
035716.png,0
035717.png,5
035718.png,4
035719.png,4
035720.png,2
035721.png,0
035722.png,7
035723.png,1
035724.png,4
035725.png,8
035726.png,7
035727.png,7
035728.png,5
035729.png,1
035730.png,1
035731.png,8
035732.png,0
035733.png,4
035734.png,0
035735.png,3
035736.png,6
035737.png,8
035738.png,5
035739.png,4
035740.png,1
035741.png,6
035742.png,5
035743.png,2
035744.png,5
035745.png,4
035746.png,1
035747.png,4
035748.png,6
035749.png,5
035750.png,1
035751.png,8
035752.png,2
035753.png,8
035754.png,7
035755.png,3
035756.png,8
035757.png,2
035758.png,2
035759.png,5
035760.png,1
035761.png,4
035762.png,3
035763.png,5
035764.png,8
035765.png,6
035766.png,2
035767.png,7
035768.png,5
035769.png,3
035770.png,3
035771.png,2
035772.png,3
035773.png,6
035774.png,7
035775.png,3
035776.png,8
035777.png,6
035778.png,4
035779.png,1

036719.png,2
036720.png,7
036721.png,4
036722.png,7
036723.png,3
036724.png,0
036725.png,3
036726.png,8
036727.png,3
036728.png,4
036729.png,8
036730.png,7
036731.png,6
036732.png,3
036733.png,5
036734.png,5
036735.png,3
036736.png,1
036737.png,7
036738.png,4
036739.png,1
036740.png,1
036741.png,4
036742.png,6
036743.png,8
036744.png,3
036745.png,1
036746.png,6
036747.png,2
036748.png,4
036749.png,4
036750.png,3
036751.png,8
036752.png,8
036753.png,4
036754.png,1
036755.png,3
036756.png,7
036757.png,7
036758.png,0
036759.png,4
036760.png,0
036761.png,6
036762.png,6
036763.png,6
036764.png,7
036765.png,5
036766.png,4
036767.png,0
036768.png,0
036769.png,8
036770.png,4
036771.png,0
036772.png,7
036773.png,8
036774.png,2
036775.png,2
036776.png,1
036777.png,1
036778.png,0
036779.png,7
036780.png,2
036781.png,0
036782.png,7
036783.png,3
036784.png,0
036785.png,5
036786.png,5
036787.png,5
036788.png,2
036789.png,2
036790.png,0
036791.png,2
036792.png,2
036793.png,4
036794.png,0
036795.png,8

037583.png,8
037584.png,6
037585.png,7
037586.png,5
037587.png,0
037588.png,4
037589.png,8
037590.png,1
037591.png,0
037592.png,0
037593.png,6
037594.png,2
037595.png,0
037596.png,5
037597.png,8
037598.png,2
037599.png,8
037600.png,4
037601.png,2
037602.png,7
037603.png,6
037604.png,5
037605.png,8
037606.png,5
037607.png,2
037608.png,8
037609.png,4
037610.png,6
037611.png,8
037612.png,2
037613.png,3
037614.png,5
037615.png,4
037616.png,3
037617.png,6
037618.png,3
037619.png,4
037620.png,2
037621.png,3
037622.png,2
037623.png,2
037624.png,0
037625.png,0
037626.png,7
037627.png,0
037628.png,5
037629.png,4
037630.png,7
037631.png,5
037632.png,0
037633.png,1
037634.png,5
037635.png,3
037636.png,0
037637.png,2
037638.png,7
037639.png,0
037640.png,7
037641.png,8
037642.png,2
037643.png,5
037644.png,2
037645.png,5
037646.png,6
037647.png,7
037648.png,3
037649.png,8
037650.png,6
037651.png,1
037652.png,3
037653.png,3
037654.png,3
037655.png,2
037656.png,3
037657.png,3
037658.png,3
037659.png,4

038611.png,6
038612.png,0
038613.png,2
038614.png,7
038615.png,6
038616.png,4
038617.png,1
038618.png,2
038619.png,7
038620.png,8
038621.png,1
038622.png,2
038623.png,6
038624.png,5
038625.png,2
038626.png,5
038627.png,0
038628.png,5
038629.png,1
038630.png,4
038631.png,4
038632.png,3
038633.png,1
038634.png,2
038635.png,3
038636.png,0
038637.png,1
038638.png,0
038639.png,0
038640.png,6
038641.png,0
038642.png,8
038643.png,4
038644.png,5
038645.png,7
038646.png,3
038647.png,2
038648.png,2
038649.png,6
038650.png,0
038651.png,4
038652.png,3
038653.png,6
038654.png,1
038655.png,0
038656.png,8
038657.png,3
038658.png,7
038659.png,2
038660.png,2
038661.png,0
038662.png,4
038663.png,7
038664.png,6
038665.png,6
038666.png,5
038667.png,0
038668.png,6
038669.png,3
038670.png,3
038671.png,5
038672.png,8
038673.png,8
038674.png,2
038675.png,2
038676.png,7
038677.png,5
038678.png,3
038679.png,7
038680.png,0
038681.png,3
038682.png,1
038683.png,2
038684.png,8
038685.png,5
038686.png,1
038687.png,2

039535.png,2
039536.png,1
039537.png,0
039538.png,2
039539.png,7
039540.png,3
039541.png,2
039542.png,0
039543.png,2
039544.png,8
039545.png,7
039546.png,8
039547.png,0
039548.png,2
039549.png,5
039550.png,3
039551.png,8
039552.png,5
039553.png,3
039554.png,1
039555.png,5
039556.png,1
039557.png,5
039558.png,5
039559.png,1
039560.png,7
039561.png,0
039562.png,3
039563.png,0
039564.png,2
039565.png,6
039566.png,0
039567.png,3
039568.png,6
039569.png,8
039570.png,8
039571.png,1
039572.png,4
039573.png,2
039574.png,4
039575.png,0
039576.png,3
039577.png,3
039578.png,8
039579.png,7
039580.png,5
039581.png,2
039582.png,6
039583.png,7
039584.png,2
039585.png,8
039586.png,2
039587.png,1
039588.png,4
039589.png,5
039590.png,8
039591.png,2
039592.png,2
039593.png,8
039594.png,5
039595.png,1
039596.png,2
039597.png,3
039598.png,8
039599.png,6
039600.png,3
039601.png,8
039602.png,6
039603.png,0
039604.png,0
039605.png,7
039606.png,4
039607.png,1
039608.png,0
039609.png,3
039610.png,1
039611.png,0